In [ ]:
import googleapiclient.discovery
import csv
import re
from urllib.parse import urlparse, parse_qs

api_key = ""

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

In [ ]:

def convert_duration_to_min_sec(duration):
    # Use regex to extract minutes and seconds
    pattern = re.compile(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?')
    match = pattern.match(duration)

    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0

    # Format the output based on whether hours are present
    if hours > 0:
        return f"{hours}:{minutes:02d}:{seconds:02d}"  # H:MM:SS format
    else:
        return f"{minutes}:{seconds:02d}"  # MM:SS format


def getinfo(video_ids):
    
    csv_file = "video_info_latest.csv"
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Video ID", "Made for Kids", "Duration", "Language"])  # Write header

    for video_id in video_ids:
        request = youtube.videos().list(part="contentDetails, status, snippet", id=video_id)
        response = request.execute()


        try:
            is_made_for_kids = response['items'][0]['status']['madeForKids']
        except Exception:
            is_made_for_kids = "ERROR"

        try:
            lang = response['items'][0]['snippet']['defaultAudioLanguage']
        except Exception:
            lang = "ERROR"

        # try:
        #     desc = response['items'][0]['snippet']['description']
        # except Exception:
        #     desc = "ERROR"

        try:
            raw_duration = response['items'][0]['contentDetails']['duration']
            formatted_duration = convert_duration_to_min_sec(raw_duration)
        except Exception:
            formatted_duration = "ERROR" 

        # try:
        #     thumb = response['items'][0]['snippet']['thumbnails']
        # except Exception:
        #     thumb = "ERROR"

        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([video_id, is_made_for_kids, formatted_duration, lang])

In [ ]:
def extract_video_id(youtube_url):
    parsed_url = urlparse(youtube_url)
    
    query_params = parse_qs(parsed_url.query)
    
    video_id = query_params.get('v', [None])[0]
    
    return video_id

In [ ]:

id_list = []
with open("Count.txt", 'r') as file:
    lines = file.readlines()
    
    for line in lines:
        id_list.append(line.replace("\n", ""))

print(id_list)

getinfo(id_list)